In [18]:
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.pipeline import Pipeline

In [10]:
titanic_train = pd.read_csv("../data/train.csv")
titanic_test = pd.read_csv("../data/test.csv")

In [11]:
titanic_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

I'm taking out the names for now

In [26]:
X_train = titanic_train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y_train = titanic_train['Survived']

X_test = titanic_test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked']]

In [32]:
X_test['Pclass']

0      3
1      3
2      2
3      3
4      3
5      3
6      3
7      2
8      3
9      3
10     3
11     1
12     1
13     2
14     1
15     2
16     2
17     3
18     3
19     3
20     1
21     3
22     1
23     1
24     1
25     3
26     1
27     3
28     1
29     3
      ..
388    3
389    3
390    1
391    1
392    3
393    2
394    3
395    1
396    3
397    1
398    3
399    3
400    1
401    2
402    1
403    1
404    1
405    2
406    2
407    1
408    3
409    3
410    3
411    1
412    3
413    3
414    1
415    3
416    3
417    3
Name: Pclass, dtype: int64

In [31]:
label_encoder = LabelEncoder()
X_train['Pclass'] = label_encoder.fit_transform(X_train['Pclass'])
X_test['Pclass'] = label_encoder.transform(X_test['Pclass'])

/home/deadhand/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


ValueError: y contains new labels: [3]

In [19]:
?LabelEncoder

In [13]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
# Need Drop first for to remove multicolinearity

In [14]:
mean_imputer = Imputer(strategy='mean')
X_train_imputed = mean_imputer.fit_transform(X_train)
X_test_imputed = mean_imputer.transform(X_test)
# Would use MICE, but I'm getting compiler errors

# Let's get a baseline

In [17]:
ridge = RidgeClassifier()
accuracy_scorer = make_scorer(accuracy_score)
cross_val_score(ridge, X_train_imputed, y_train, scoring=accuracy_scorer, cv = 10).mean()
# This is decent enough baseline

0.79124304846214966